<h1> <center> 7 - Co-voting networks </center> </h1>

**Author:** Sebastião Rosalino, nº 20230372

**Supervisors:** Flávio Pinheiro, António Curado

<div style="text-align: center;">
    <img src="../logos/nova_ims_logo.png" alt="Image 1" style="width: 15%; display: inline-block; margin: 0 10px;"/>
    <img src="../logos/european_parliament_logo.png" alt="Image 2" style="width: 15%; display: inline-block; margin: 0 10px;"/>
</div>

# Table of Contents

1. [All bills](#All-bills)
2. [Years](#Years)
   - 2.1. [2019](#2019)
   - 2.2. [2020](#2020)
   - 2.3. [2021](#2021)
   - 2.4. [2022](#2022)
   - 2.5. [2023](#2023)
   - 2.6. [2024](#2024)
3. [Primary subjects](#Primary-subjects)
   - 3.1. [European citizenship](#European-citizenship)
   - 3.2. [Internal market, single market](#Internal-market-single-market)
   - 3.3. [Community policies](#Community-policies)
   - 3.4. [Economic, social and territorial cohesion](#Economic-social-and-territorial-cohesion)
   - 3.5. [Economic and monetary system](#Economic-and-monetary-system)
   - 3.6. [External relations of the Union](#External-relations-of-the-Union)
   - 3.7. [Area of freedom, security and justice](#Area-of-freedom-security-and-justice)
   - 3.8. [State and evolution of the Union](#State-and-evolution-of-the-Union)

In [821]:
# Import the necessary libraries and configurations
import pandas as pd
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

In [822]:
# Read all the datasets from their corresponding folders
members = pd.read_csv("../datasets/second_stage/2_members/members.csv")
group_memberships_single = pd.read_csv("../datasets/second_stage/3_group_memberships/group_memberships_single.csv")
bills = pd.read_csv("../datasets/second_stage/4_bills/bills.csv")
member_votes = pd.read_csv("../datasets/second_stage/5_member_votes/member_votes.csv")
primary_subject_bills = pd.read_csv("../datasets/second_stage/6_bill_subject_assignment/primary_subject_bills.csv")
secondary_subject_bills = pd.read_csv("../datasets/second_stage/6_bill_subject_assignment/secondary_subject_bills.csv")
tertiary_subject_bills = pd.read_csv("../datasets/second_stage/6_bill_subject_assignment/tertiary_subject_bills.csv")

In [823]:
# A reminder of how the members dataset looks like
members

,country_code,id_mep,name,full_term
0,LUX,0,Charles GOERENS,1
1,GBR,1,Bill NEWTON DUNN,0
2,DEU,2,Constanze KREHL,0
3,PRT,3,Carlos COELHO,0
4,DEU,4,Bernd LANGE,1
...,...,...,...,...
866,PRT,868,Vasco BECKER-WEINBERG,0
867,PRT,869,Ana Miguel DOS SANTOS,0
868,PRT,870,Vânia NETO,0
869,PRT,871,Teófilo SANTOS,0


In [824]:
# A reminder of how the group_memberships_single looks like
group_memberships_single

,group_code,start_date,end_date,id_mep,membership_length
0,RENEW,2019-07-02,2024-07-15,0,1840
1,RENEW,2019-07-02,2020-01-31,1,213
2,SD,2019-07-02,2022-10-02,2,1188
3,EPP,2023-07-07,2024-07-15,3,374
4,SD,2019-07-02,2024-07-15,4,1840
...,...,...,...,...,...
866,EPP,2024-03-26,2024-07-15,868,111
867,EPP,2024-04-02,2024-07-15,869,104
868,EPP,2024-04-02,2024-07-15,870,104
869,EPP,2024-04-02,2024-07-15,871,104


In [825]:
# A reminder of how the member_votes dataset looks like
member_votes

,position,country_code,group_code,id_mep,id_bill
0,FOR,LUX,RENEW,0,0
1,FOR,GBR,RENEW,1,0
2,DID_NOT_VOTE,DEU,SD,2,0
3,FOR,DEU,SD,4,0
4,FOR,DEU,SD,5,0
...,...,...,...,...,...
1345669,DID_NOT_VOTE,PRT,EPP,868,1902
1345670,DID_NOT_VOTE,PRT,EPP,869,1902
1345671,DID_NOT_VOTE,PRT,EPP,870,1902
1345672,FOR,PRT,EPP,871,1902


In [826]:
def create_agents_and_artifacts_matrix(member_votes):
    """
    Create a matrix where rows represent MEPs (agents) and columns represent bills (artifacts), and values are 1 if the MEP voted "FOR" on a bill, otherwise 0.

    Parameters:
    member_votes (DataFrame): A dataset containing voting records with columns
                              'position', 'id_mep', and 'id_bill'.
    members (DataFrame): A dataset containing MEP information with columns
                         'id_mep', 'name', and 'country_code'.
    group_memberships_single (DataFrame): A dataset containing MEP group memberships with columns
                                          'id_mep' and 'group_code'.

    Returns:
    DataFrame: A matrix with MEPs as rows, bills as columns, and binary values for voting positions.
    """
    # Filter votes where position is 'FOR'
    member_votes['vote_for'] = (member_votes['position'] == 'FOR').astype(int)

    # Pivot the dataset to create the matrix
    votes_matrix = member_votes.pivot_table(
        index='id_mep',     # Rows are MEPs (agents)
        columns='id_bill',  # Columns are bills (artifacts)
        values='vote_for',  # Values are 1 if the position is 'FOR', otherwise 0
        fill_value=0        # Fill missing values (MEPs that weren't serving at the EP at the time of the bill) with 0
    )

    # Convert matrix values to integer type
    votes_matrix = votes_matrix.astype(int)

    # Reset the column names to 'bill_1', 'bill_2', ...
    votes_matrix.columns = [f"bill_{int(col)+1}" for col in votes_matrix.columns]

    # Retrieve the columns "id_mep", "name" and "country_code" from the members dataset into the members_info dataframe
    members_info = members[['id_mep', 'name', 'country_code']]

    # Merge the members_info dataframe with the group_memberships_single dataset to get the group of each MEP
    members_info = pd.merge(members_info, group_memberships_single[['id_mep', 'group_code']], on='id_mep', how='left')

    # Create the formatted name column and rename it to "name". The formatted name column will be: Name (Country-Group)
    members_info['name'] = members_info.apply(
        lambda x: f"{x['name']} ({x['country_code']}-{x['group_code']})", axis=1
    )

    # Merge the formatted name column with the votes matrix
    votes_matrix = votes_matrix.reset_index()
    votes_matrix = pd.merge(votes_matrix, members_info[['id_mep', 'name']], on='id_mep', how='left')

    # Drop the id_mep column
    votes_matrix = votes_matrix.drop(columns=['id_mep'])

    # Reorder the votes_matrix columns so that the name column comes first
    votes_matrix = votes_matrix[['name'] + [col for col in votes_matrix.columns if col != 'name']]
    
    return votes_matrix

In [827]:
def create_gephi_edges_file(edgelist):
    """
    Cleans and processes a backbone edgelist for Gephi compatibility by:
    - Dropping the unnecessary "weight" column.
    - Renaming columns to "Source" and "Target".
    - Removing country and group information from names coming from the agents and artifacts matrix.
    - Replacing names with MEP IDs based on the members dataset.

    Parameters:
        edgelist (pd.DataFrame): The raw backbone edgelist.

    Returns:
        pd.DataFrame: A cleaned Gephi-compatible edges file with MEP IDs as Source and Target.
    """

    # Drop the unnecessary "weight" column
    edgelist.drop("weight", inplace=True, axis=1)
    
    # Rename columns for Gephi compatibility
    edgelist = edgelist.rename(columns={"from": "Source", "to": "Target"})

    # Remove country and group information from names coming from the agents and artifacts matrix
    def clean_name(name):
        return name.split(' (')[0]

    edgelist["Source"] = edgelist["Source"].apply(clean_name)
    edgelist["Target"] = edgelist["Target"].apply(clean_name)

    # Replace names with ids using the members dataset
    name_to_id = members.set_index("name")["id_mep"].to_dict()

    edgelist["Source"] = edgelist["Source"].map(name_to_id)
    edgelist["Target"] = edgelist["Target"].map(name_to_id)

    return edgelist

## 1. All bills
<a id="All-bills"></a>

In [828]:
# Create the agents and artifacts matrix for all bills
all_bills = create_agents_and_artifacts_matrix(member_votes)

all_bills

,name,bill_1,bill_2,bill_3,bill_4,bill_5,bill_6,bill_7,bill_8,bill_9,...,bill_1894,bill_1895,bill_1896,bill_1897,bill_1898,bill_1899,bill_1900,bill_1901,bill_1902,bill_1903
0,Charles GOERENS (LUX-RENEW),1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,1,1,1,1,1
4,Bernd LANGE (DEU-SD),1,0,0,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
866,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
867,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
868,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,1,1,1


In [829]:
# Save the agents and artifacts matrix for all bills
all_bills.to_csv("../matrices_and_networks/all_bills/agents_and_artifacts_all_bills.csv", index=False)

In [830]:
# Load the edgelist for all bills
edgelist_all_bills = pd.read_csv("../matrices_and_networks/all_bills/edgelist_all_bills.csv")

In [831]:
# Visualize the edgelist for all bills
edgelist_all_bills

,from,to,weight
0,Charles GOERENS (LUX-RENEW),Peter LIESE (DEU-EPP),1
1,Charles GOERENS (LUX-RENEW),José Manuel GARCÍA-MARGALLO Y MARFIL (ESP-EPP),1
2,Charles GOERENS (LUX-RENEW),Frédérique RIES (BEL-RENEW),1
3,Charles GOERENS (LUX-RENEW),Angelika NIEBLER (DEU-EPP),1
4,Charles GOERENS (LUX-RENEW),María Soraya RODRÍGUEZ RAMOS (ESP-RENEW),1
...,...,...,...
64418,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
64419,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
64420,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
64421,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [832]:
# Prepare the edges file of all bills for Gephi
edges_all_bills = create_gephi_edges_file(edgelist_all_bills)

# Export the edges file as a CSV
edges_all_bills.to_csv("../matrices_and_networks/all_bills/edges.csv", index=False)

In [833]:
# Filter MEPs present in either Source or Target in edges_all_bills
unique_mep_ids = set(edges_all_bills['Source']).union(set(edges_all_bills['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/all_bills/nodes.csv", index=False)

## 2. Years
<a id="#Years"></a>

### 2.1. 2019
<a id="#2019"></a>

In [834]:
# Ensure the 'timestamp' column is in datetime format
bills['timestamp'] = pd.to_datetime(bills['timestamp'])

# Filter for bills from the year 2019
bills_2019 = bills[bills['timestamp'].dt.year == 2019]

In [835]:
# Filter the member_votes dataset to include only voting records corresponding to bills from 2019
member_votes_2019 = member_votes[member_votes['id_bill'].isin(bills_2019['id_bill'])]

In [836]:
# Create the agents and artifacts matrix for all bills from 2019
all_bills_2019 = create_agents_and_artifacts_matrix(member_votes_2019)

all_bills_2019

,name,bill_1,bill_2,bill_3,bill_4,bill_5,bill_6,bill_7,bill_8,bill_9,...,bill_80,bill_81,bill_82,bill_83,bill_84,bill_85,bill_86,bill_87,bill_88,bill_89
0,Charles GOERENS (LUX-RENEW),1,1,1,1,1,1,1,1,1,...,1,0,0,1,1,1,1,1,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,0,1
2,Constanze KREHL (DEU-SD),0,0,0,1,1,1,1,1,1,...,1,0,1,1,0,0,0,0,0,0
3,Bernd LANGE (DEU-SD),1,0,0,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,0,1
4,Evelyne GEBHARDT (DEU-SD),1,0,0,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Sunčana GLAVAK (HRV-EPP),0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
752,Marc ANGEL (LUX-SD),0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1
753,Gheorghe-Vlad NISTOR (ROU-EPP),0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1
754,Carles PUIGDEMONT I CASAMAJÓ (ESP-NI),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [837]:
# Save the agents and artifacts matrix for all bills from 2019
all_bills_2019.to_csv("../matrices_and_networks/years/2019/agents_and_artifacts_2019.csv", index=False)

In [838]:
# Load the edgelist for all bills from 2019
edgelist_2019 = pd.read_csv("../matrices_and_networks/years/2019/edgelist_2019.csv")

In [839]:
# Visualize the edgelist for all bills from 2019
edgelist_2019

,from,to,weight
0,Constanze KREHL (DEU-SD),Miriam DALLI (MLT-SD),1
1,Constanze KREHL (DEU-SD),Sylwia SPUREK (POL-GREEN_EFA),1
2,Markus FERBER (DEU-EPP),Peter LIESE (DEU-EPP),1
3,Markus FERBER (DEU-EPP),José Manuel GARCÍA-MARGALLO Y MARFIL (ESP-EPP),1
4,Markus FERBER (DEU-EPP),Rainer WIELAND (DEU-EPP),1
...,...,...,...
4927,Liudas MAŽYLIS (LTU-EPP),Andrius KUBILIUS (LTU-EPP),1
4928,Andrius KUBILIUS (LTU-EPP),Maria WALSH (IRL-EPP),1
4929,Sunčana GLAVAK (HRV-EPP),Marc ANGEL (LUX-SD),1
4930,Sunčana GLAVAK (HRV-EPP),Gheorghe-Vlad NISTOR (ROU-EPP),1


In [840]:
# Prepare the edges file of all bills from 2019 for Gephi
edges_2019 = create_gephi_edges_file(edgelist_2019)

# Export the edges file as a CSV
edges_2019.to_csv("../matrices_and_networks/years/2019/edges.csv", index=False)

In [841]:
# Filter MEPs present in either Source or Target in edges_2019
unique_mep_ids = set(edges_2019['Source']).union(set(edges_2019['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/years/2019/nodes.csv", index=False)

### 2.2. 2020
<a id="#2020"></a>

In [842]:
# Filter for bills from the year 2020
bills_2020 = bills[bills['timestamp'].dt.year == 2020]

In [843]:
# Filter the member_votes dataset to include only voting records corresponding to bills from 2020
member_votes_2020 = member_votes[member_votes['id_bill'].isin(bills_2020['id_bill'])]

In [844]:
# Create the agents and artifacts matrix for all bills from 2020
all_bills_2020 = create_agents_and_artifacts_matrix(member_votes_2020)

all_bills_2020

,name,bill_90,bill_91,bill_92,bill_93,bill_94,bill_95,bill_96,bill_97,bill_98,...,bill_381,bill_382,bill_383,bill_384,bill_385,bill_386,bill_387,bill_388,bill_389,bill_390
0,Charles GOERENS (LUX-RENEW),1,0,1,1,1,1,0,1,0,...,1,1,1,1,0,0,0,0,0,1
1,Bill NEWTON DUNN (GBR-RENEW),1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),0,1,1,1,1,0,0,1,1,...,1,1,1,1,1,1,1,1,1,0
3,Bernd LANGE (DEU-SD),0,1,1,1,1,0,0,1,0,...,1,1,1,1,1,1,1,1,1,1
4,Evelyne GEBHARDT (DEU-SD),1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,Manuela RIPA (DEU-GREEN_EFA),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
780,Sara MATTHIEU (BEL-GREEN_EFA),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
781,Cyrus ENGERER (MLT-SD),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
782,Vlad GHEORGHE (ROU-RENEW),0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,1,1,1,1


In [845]:
# Save the agents and artifacts matrix for all bills from 2020
all_bills_2020.to_csv("../matrices_and_networks/years/2020/agents_and_artifacts_2020.csv", index=False)

In [846]:
# Load the edgelist for all bills from 2020
edgelist_2020 = pd.read_csv("../matrices_and_networks/years/2020/edgelist_2020.csv")

In [847]:
# Visualize the edgelist for all bills from 2020
edgelist_2020

,from,to,weight
0,Charles GOERENS (LUX-RENEW),María Soraya RODRÍGUEZ RAMOS (ESP-RENEW),1
1,Charles GOERENS (LUX-RENEW),Maite PAGAZAURTUNDÚA (ESP-RENEW),1
2,Charles GOERENS (LUX-RENEW),Jordi CAÑAS (ESP-RENEW),1
3,Charles GOERENS (LUX-RENEW),Hilde VAUTMANS (BEL-RENEW),1
4,Charles GOERENS (LUX-RENEW),Monica SEMEDO (LUX-RENEW),1
...,...,...,...
18490,Sara MATTHIEU (BEL-GREEN_EFA),Vlad GHEORGHE (ROU-RENEW),1
18491,Sara MATTHIEU (BEL-GREEN_EFA),Colm MARKEY (IRL-EPP),1
18492,Cyrus ENGERER (MLT-SD),Vlad GHEORGHE (ROU-RENEW),1
18493,Cyrus ENGERER (MLT-SD),Colm MARKEY (IRL-EPP),1


In [848]:
# Prepare the edges file of all bills from 2020 for Gephi
edges_2020 = create_gephi_edges_file(edgelist_2020)

# Export the edges file as a CSV
edges_2020.to_csv("../matrices_and_networks/years/2020/edges.csv", index=False)

In [849]:
# Filter MEPs present in either Source or Target in edges_2020
unique_mep_ids = set(edges_2020['Source']).union(set(edges_2020['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/years/2020/nodes.csv", index=False)

### 2.3. 2021
<a id="#2021"></a>

In [850]:
# Filter for bills from the year 2021
bills_2021 = bills[bills['timestamp'].dt.year == 2021]

In [851]:
# Filter the member_votes dataset to include only voting records corresponding to bills from 2021
member_votes_2021 = member_votes[member_votes['id_bill'].isin(bills_2021['id_bill'])]

In [852]:
# Create the agents and artifacts matrix for all bills from 2021
all_bills_2021 = create_agents_and_artifacts_matrix(member_votes_2021)

all_bills_2021

,name,bill_391,bill_392,bill_393,bill_394,bill_395,bill_396,bill_397,bill_398,bill_399,...,bill_807,bill_808,bill_809,bill_810,bill_811,bill_812,bill_813,bill_814,bill_815,bill_816
0,Charles GOERENS (LUX-RENEW),1,1,1,1,1,1,1,1,0,...,1,1,0,1,1,1,1,1,1,1
1,Constanze KREHL (DEU-SD),1,1,1,1,1,1,1,1,0,...,1,1,0,1,1,1,0,1,1,1
2,Bernd LANGE (DEU-SD),1,1,1,1,1,0,0,0,0,...,1,1,0,1,1,1,0,1,1,1
3,Evelyne GEBHARDT (DEU-SD),1,1,1,1,1,1,1,1,0,...,1,1,0,1,1,1,0,1,1,1
4,Markus FERBER (DEU-EPP),1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,Thijs REUTEN (NLD-SD),0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,1
707,Michiel HOOGEVEEN (NLD-ECR),0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,0
708,Ladislav ILČIĆ (HRV-ECR),0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,1,1,0,1
709,Karolin BRAUNSBERGER-REINHOLD (DEU-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,1,1,1,1


In [853]:
# Save the agents and artifacts matrix for all bills from 2021
all_bills_2021.to_csv("../matrices_and_networks/years/2021/agents_and_artifacts_2021.csv", index=False)

In [854]:
# Load the edgelist for all bills from 2021
edgelist_2021 = pd.read_csv("../matrices_and_networks/years/2021/edgelist_2021.csv")

In [855]:
# Visualize the edgelist for all bills from 2020
edgelist_2021

,from,to,weight
0,Charles GOERENS (LUX-RENEW),Sophia IN 'T VELD (NLD-RENEW),1
1,Charles GOERENS (LUX-RENEW),Paolo DE CASTRO (ITA-SD),1
2,Charles GOERENS (LUX-RENEW),Guy VERHOFSTADT (BEL-RENEW),1
3,Charles GOERENS (LUX-RENEW),Nils TORVALDS (FIN-RENEW),1
4,Charles GOERENS (LUX-RENEW),Andrus ANSIP (EST-RENEW),1
...,...,...,...
21311,Clara PONSATÍ OBIOLS (ESP-NI),Sara MATTHIEU (BEL-GREEN_EFA),1
21312,Manuela RIPA (DEU-GREEN_EFA),Sara MATTHIEU (BEL-GREEN_EFA),1
21313,Vlad GHEORGHE (ROU-RENEW),Alin MITUȚA (ROU-RENEW),1
21314,Ladislav ILČIĆ (HRV-ECR),Karolin BRAUNSBERGER-REINHOLD (DEU-EPP),1


In [856]:
# Prepare the edges file of all bills from 2021 for Gephi
edges_2021 = create_gephi_edges_file(edgelist_2021)

# Export the edges file as a CSV
edges_2021.to_csv("../matrices_and_networks/years/2021/edges.csv", index=False)

In [857]:
# Filter MEPs present in either Source or Target in edges_2021
unique_mep_ids = set(edges_2021['Source']).union(set(edges_2021['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/years/2021/nodes.csv", index=False)

### 2.4. 2022
<a id="#2022"></a>

In [858]:
# Filter for bills from the year 2022
bills_2022 = bills[bills['timestamp'].dt.year == 2022]

In [859]:
# Filter the member_votes dataset to include only voting records corresponding to bills from 2022
member_votes_2022 = member_votes[member_votes['id_bill'].isin(bills_2022['id_bill'])]

In [860]:
# Create the agents and artifacts matrix for all bills from 2022
all_bills_2022 = create_agents_and_artifacts_matrix(member_votes_2022)

all_bills_2022

,name,bill_817,bill_818,bill_819,bill_820,bill_821,bill_822,bill_823,bill_824,bill_825,...,bill_1183,bill_1184,bill_1185,bill_1186,bill_1187,bill_1188,bill_1189,bill_1190,bill_1191,bill_1192
0,Charles GOERENS (LUX-RENEW),1,1,1,1,1,1,1,1,1,...,0,1,1,1,1,0,0,1,1,1
1,Constanze KREHL (DEU-SD),1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Bernd LANGE (DEU-SD),1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,0,1,1,1
3,Evelyne GEBHARDT (DEU-SD),1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Markus FERBER (DEU-EPP),1,0,0,1,1,1,1,1,1,...,0,1,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,Elisabetta DE BLASIS (ITA-ID),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
728,Eleni STAVROU (CYP-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,0
729,Erik POULSEN (DNK-RENEW),0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,1,1,1
730,Bergur Løkke RASMUSSEN (DNK-RENEW),0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,1,1,1


In [861]:
# Save the agents and artifacts matrix for all bills from 2022
all_bills_2022.to_csv("../matrices_and_networks/years/2022/agents_and_artifacts_2022.csv", index=False)

In [862]:
# Load the edgelist for all bills from 2022
edgelist_2022 = pd.read_csv("../matrices_and_networks/years/2022/edgelist_2022.csv")

In [863]:
# Visualize the edgelist for all bills from 2022
edgelist_2022

,from,to,weight
0,Charles GOERENS (LUX-RENEW),Sophia IN 'T VELD (NLD-RENEW),1
1,Charles GOERENS (LUX-RENEW),Juan Fernando LÓPEZ AGUILAR (ESP-SD),1
2,Charles GOERENS (LUX-RENEW),Jana TOOM (EST-RENEW),1
3,Charles GOERENS (LUX-RENEW),Monica SEMEDO (LUX-RENEW),1
4,Charles GOERENS (LUX-RENEW),Ivars IJABS (LVA-RENEW),1
...,...,...,...
14627,Eleni STAVROU (CYP-EPP),Bergur Løkke RASMUSSEN (DNK-RENEW),1
14628,Eleni STAVROU (CYP-EPP),Beatrice COVASSI (ITA-SD),1
14629,Erik POULSEN (DNK-RENEW),Bergur Løkke RASMUSSEN (DNK-RENEW),1
14630,Erik POULSEN (DNK-RENEW),Beatrice COVASSI (ITA-SD),1


In [864]:
# Prepare the edges file of all bills from 2022 for Gephi
edges_2022 = create_gephi_edges_file(edgelist_2022)

# Export the edges file as a CSV
edges_2022.to_csv("../matrices_and_networks/years/2022/edges.csv", index=False)

In [865]:
# Filter MEPs present in either Source or Target in edges_2022
unique_mep_ids = set(edges_2022['Source']).union(set(edges_2022['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/years/2022/nodes.csv", index=False)

### 2.5. 2023
<a id="#2023"></a>

In [866]:
# Filter for bills from the year 2023
bills_2023 = bills[bills['timestamp'].dt.year == 2023]

In [867]:
# Filter the member_votes dataset to include only voting records corresponding to bills from 2023
member_votes_2023 = member_votes[member_votes['id_bill'].isin(bills_2023['id_bill'])]

In [868]:
# Create the agents and artifacts matrix for all bills from 2023
all_bills_2023 = create_agents_and_artifacts_matrix(member_votes_2023)

all_bills_2023

,name,bill_1193,bill_1194,bill_1195,bill_1196,bill_1197,bill_1198,bill_1199,bill_1200,bill_1201,...,bill_1593,bill_1594,bill_1595,bill_1596,bill_1597,bill_1598,bill_1599,bill_1600,bill_1601,bill_1602
0,Charles GOERENS (LUX-RENEW),1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0
1,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,1,1,1,1,1
2,Bernd LANGE (DEU-SD),1,1,0,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,0,1
3,Markus FERBER (DEU-EPP),1,1,1,1,0,1,1,1,1,...,0,0,1,1,1,1,1,1,1,1
4,Peter LIESE (DEU-EPP),0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,Witold PAHL (POL-EPP),0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,1,1,1,1,1
723,Włodzimierz KARPIŃSKI (POL-EPP),0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,1,1,1,1,1
724,Rafał ROMANOWSKI (POL-ECR),0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,1,1,1,1,1
725,Patricia CARO MAYA (ESP-GUE_NGL),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [869]:
# Save the agents and artifacts matrix for all bills from 2023
all_bills_2023.to_csv("../matrices_and_networks/years/2023/agents_and_artifacts_2023.csv", index=False)

In [870]:
# Load the edgelist for all bills from 2023
edgelist_2023 = pd.read_csv("../matrices_and_networks/years/2023/edgelist_2023.csv")

In [871]:
# Visualize the edgelist for all bills from 2023
edgelist_2023

,from,to,weight
0,Charles GOERENS (LUX-RENEW),Marie-Pierre VEDRENNE (FRA-RENEW),1
1,Charles GOERENS (LUX-RENEW),Salima YENBOU (FRA-GREEN_EFA),1
2,Charles GOERENS (LUX-RENEW),Lídia PEREIRA (PRT-EPP),1
3,Charles GOERENS (LUX-RENEW),Marc ANGEL (LUX-SD),1
4,Charles GOERENS (LUX-RENEW),Erik POULSEN (DNK-RENEW),1
...,...,...,...
12475,Witold PAHL (POL-EPP),Rafał ROMANOWSKI (POL-ECR),1
12476,Witold PAHL (POL-EPP),François THIOLLET (FRA-GREEN_EFA),1
12477,Włodzimierz KARPIŃSKI (POL-EPP),Rafał ROMANOWSKI (POL-ECR),1
12478,Włodzimierz KARPIŃSKI (POL-EPP),François THIOLLET (FRA-GREEN_EFA),1


In [872]:
# Prepare the edges file of all bills from 2023 for Gephi
edges_2023 = create_gephi_edges_file(edgelist_2023)

# Export the edges file as a CSV
edges_2023.to_csv("../matrices_and_networks/years/2023/edges.csv", index=False)

In [873]:
# Filter MEPs present in either Source or Target in edges_2023
unique_mep_ids = set(edges_2023['Source']).union(set(edges_2023['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/years/2023/nodes.csv", index=False)

### 2.6. 2024
<a id="#2024"></a>

In [874]:
# Filter for bills from the year 2024
bills_2024 = bills[bills['timestamp'].dt.year == 2024]

In [875]:
# Filter the member_votes dataset to include only voting records corresponding to bills from 2024
member_votes_2024 = member_votes[member_votes['id_bill'].isin(bills_2024['id_bill'])]

In [876]:
# Create the agents and artifacts matrix for all bills from 2024
all_bills_2024 = create_agents_and_artifacts_matrix(member_votes_2024)

all_bills_2024

,name,bill_1576,bill_1603,bill_1604,bill_1605,bill_1606,bill_1607,bill_1608,bill_1609,bill_1610,...,bill_1894,bill_1895,bill_1896,bill_1897,bill_1898,bill_1899,bill_1900,bill_1901,bill_1902,bill_1903
0,Charles GOERENS (LUX-RENEW),1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,Carlos COELHO (PRT-EPP),1,1,1,1,1,1,1,1,1,...,1,0,0,1,1,1,1,1,1,1
2,Bernd LANGE (DEU-SD),0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1
3,Markus FERBER (DEU-EPP),1,1,1,1,1,1,1,1,1,...,1,0,0,1,1,1,1,1,1,1
4,Peter LIESE (DEU-EPP),1,1,1,1,1,1,1,1,1,...,1,0,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
710,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
711,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
712,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,1,1,1,1,1


In [877]:
# Save the agents and artifacts matrix for all bills from 2024
all_bills_2024.to_csv("../matrices_and_networks/years/2024/agents_and_artifacts_2024.csv", index=False)

In [878]:
# Load the edgelist for all bills from 2024
edgelist_2024 = pd.read_csv("../matrices_and_networks/years/2024/edgelist_2024.csv")

In [879]:
# Visualize the edgelist for all bills from 2024
edgelist_2024

,from,to,weight
0,Carlos COELHO (PRT-EPP),Lídia PEREIRA (PRT-EPP),1
1,Bernd LANGE (DEU-SD),Lara WOLTERS (NLD-SD),1
2,Bernd LANGE (DEU-SD),Andreas SCHWAB (DEU-EPP),1
3,Bernd LANGE (DEU-SD),Inese VAIDERE (LVA-EPP),1
4,Bernd LANGE (DEU-SD),Rovana PLUMB (ROU-SD),1
...,...,...,...
12568,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
12569,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
12570,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
12571,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [880]:
# Prepare the edges file of all bills from 2024 for Gephi
edges_2024 = create_gephi_edges_file(edgelist_2024)

# Export the edges file as a CSV
edges_2024.to_csv("../matrices_and_networks/years/2024/edges.csv", index=False)

In [881]:
# Filter MEPs present in either Source or Target in edges_2024
unique_mep_ids = set(edges_2024['Source']).union(set(edges_2024['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/years/2024/nodes.csv", index=False)

## 3. Primary subjects
<a id="#Primary-subjects"></a>

### 3.1. European citizenship
<a id="European-citizenship"></a>

In [882]:
# Filter for only bills whose primary subject is 1 (among any other possible primary subjects)
primary_subject_bills_1 = primary_subject_bills[primary_subject_bills["primary_subject"] == 1]

In [883]:
# Filter the member_votes dataset to include only voting records corresponding to bills whose primary subject is 1 (among any other possible primary subjects)
member_votes_1 = member_votes[member_votes['id_bill'].isin(primary_subject_bills_1['id_bill'])]

In [884]:
# Create the agents and artifacts matrix for all bills whose primary subject is 1 (among any other possible primary subjects)
all_bills_primary_subject_1 = create_agents_and_artifacts_matrix(member_votes_1)

all_bills_primary_subject_1

,name,bill_4,bill_95,bill_97,bill_117,bill_134,bill_208,bill_231,bill_242,bill_263,...,bill_1686,bill_1719,bill_1734,bill_1737,bill_1745,bill_1773,bill_1847,bill_1850,bill_1876,bill_1888
0,Charles GOERENS (LUX-RENEW),1,1,1,1,0,1,1,0,1,...,1,1,1,1,1,1,1,0,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),1,0,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,1,1,1
4,Bernd LANGE (DEU-SD),1,0,1,1,0,1,1,1,1,...,1,1,1,1,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
863,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
864,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
865,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,1


In [885]:
# Save the agents and artifacts matrix for all bills whose primary subject is 1 (among any other possible primary subjects)
all_bills_primary_subject_1.to_csv("../matrices_and_networks/primary_subjects/1_european_citizenship/agents_and_artifacts_1.csv", index=False)

In [886]:
# Load the edgelist for all bills whose primary subject is 1 (among any other possible primary subjects)
edgelist_1 = pd.read_csv("../matrices_and_networks/primary_subjects/1_european_citizenship/edgelist_1.csv")

In [887]:
# Visualize the edgelist for all bills whose primary subject is 1 (among any other possible primary subjects)
edgelist_1

,from,to,weight
0,Bill NEWTON DUNN (GBR-RENEW),Richard CORBETT (GBR-SD),1
1,Bill NEWTON DUNN (GBR-RENEW),Claude MORAES (GBR-SD),1
2,Bill NEWTON DUNN (GBR-RENEW),Neena GILL (GBR-SD),1
3,Bill NEWTON DUNN (GBR-RENEW),Jill EVANS (GBR-GREEN_EFA),1
4,Bill NEWTON DUNN (GBR-RENEW),Chris DAVIES (GBR-RENEW),1
...,...,...,...
6513,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
6514,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
6515,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
6516,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [888]:
# Prepare the edges file of all bills whose primary subject is 1 (among any other possible primary subjects)
edges_1 = create_gephi_edges_file(edgelist_1)

# Export the edges file as a CSV
edges_1.to_csv("../matrices_and_networks/primary_subjects/1_european_citizenship/edges.csv", index=False)

In [889]:
# Filter MEPs present in either Source or Target in edges_1
unique_mep_ids = set(edges_1['Source']).union(set(edges_1['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/primary_subjects/1_european_citizenship/nodes.csv", index=False)

### 3.2. Internal market, single market
<a id="Internal-market-single-market"></a>

In [890]:
# Filter for only bills whose primary subject is 2 (among any other possible primary subjects)
primary_subject_bills_2 = primary_subject_bills[primary_subject_bills["primary_subject"] == 2]

In [891]:
# Filter the member_votes dataset to include only voting records corresponding to bills whose primary subject is 2 (among any other possible primary subjects)
member_votes_2 = member_votes[member_votes['id_bill'].isin(primary_subject_bills_2['id_bill'])]

In [892]:
# Create the agents and artifacts matrix for all bills whose primary subject is 2 (among any other possible primary subjects)
all_bills_primary_subject_2 = create_agents_and_artifacts_matrix(member_votes_2)

all_bills_primary_subject_2

,name,bill_43,bill_51,bill_73,bill_74,bill_82,bill_93,bill_104,bill_115,bill_209,...,bill_1855,bill_1859,bill_1861,bill_1862,bill_1863,bill_1865,bill_1878,bill_1879,bill_1881,bill_1883
0,Charles GOERENS (LUX-RENEW),1,1,1,1,0,1,0,0,1,...,0,1,1,1,1,0,1,1,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,Bernd LANGE (DEU-SD),1,1,1,1,1,1,1,0,1,...,0,1,1,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,0
864,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,1
865,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,0,1,1,0,1,1,1,1,1
866,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,0,1,1,1,1


In [893]:
# Save the agents and artifacts matrix for all bills whose primary subject is 2 (among any other possible primary subjects)
all_bills_primary_subject_2.to_csv("../matrices_and_networks/primary_subjects/2_internal_market_single_market/agents_and_artifacts_2.csv", index=False)

In [894]:
# Load the edgelist for all bills whose primary subject is 2 (among any other possible primary subjects)
edgelist_2 = pd.read_csv("../matrices_and_networks/primary_subjects/2_internal_market_single_market/edgelist_2.csv")

In [895]:
# Visualize the edgelist for all bills whose primary subject is 2 (among any other possible primary subjects)
edgelist_2

,from,to,weight
0,Bill NEWTON DUNN (GBR-RENEW),Richard CORBETT (GBR-SD),1
1,Bill NEWTON DUNN (GBR-RENEW),Geoffrey VAN ORDEN (GBR-ECR),1
2,Bill NEWTON DUNN (GBR-RENEW),Claude MORAES (GBR-SD),1
3,Bill NEWTON DUNN (GBR-RENEW),Neena GILL (GBR-SD),1
4,Bill NEWTON DUNN (GBR-RENEW),Jill EVANS (GBR-GREEN_EFA),1
...,...,...,...
10113,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
10114,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
10115,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
10116,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [896]:
# Prepare the edges file of all bills whose primary subject is 2 (among any other possible primary subjects)
edges_2 = create_gephi_edges_file(edgelist_2)

# Export the edges file as a CSV
edges_2.to_csv("../matrices_and_networks/primary_subjects/2_internal_market_single_market/edges.csv", index=False)

In [897]:
# Filter MEPs present in either Source or Target in edges_2
unique_mep_ids = set(edges_2['Source']).union(set(edges_2['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/primary_subjects/2_internal_market_single_market/nodes.csv", index=False)

### 3.3. Community policies
<a id="Community-policies"></a>

In [898]:
# Filter for only bills whose primary subject is 3 (among any other possible primary subjects)
primary_subject_bills_3 = primary_subject_bills[primary_subject_bills["primary_subject"] == 3]

In [899]:
# Filter the member_votes dataset to include only voting records corresponding to bills whose primary subject is 3 (among any other possible primary subjects)
member_votes_3 = member_votes[member_votes['id_bill'].isin(primary_subject_bills_3['id_bill'])]

In [900]:
# Create the agents and artifacts matrix for all bills whose primary subject is 3 (among any other possible primary subjects)
all_bills_primary_subject_3 = create_agents_and_artifacts_matrix(member_votes_3)

all_bills_primary_subject_3

,name,bill_6,bill_18,bill_20,bill_21,bill_22,bill_23,bill_24,bill_27,bill_32,...,bill_1869,bill_1873,bill_1876,bill_1882,bill_1886,bill_1887,bill_1889,bill_1890,bill_1895,bill_1903
0,Charles GOERENS (LUX-RENEW),1,1,1,0,0,0,0,1,1,...,1,1,1,1,1,1,1,0,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,1
4,Bernd LANGE (DEU-SD),1,1,1,1,1,1,1,1,1,...,0,1,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,0
864,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,1,1,0,0,0
865,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,0
866,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,1


In [901]:
# Save the agents and artifacts matrix for all bills whose primary subject is 3 (among any other possible primary subjects)
all_bills_primary_subject_3.to_csv("../matrices_and_networks/primary_subjects/3_community_policies/agents_and_artifacts_3.csv", index=False)

In [902]:
# Load the edgelist for all bills whose primary subject is 3 (among any other possible primary subjects)
edgelist_3 = pd.read_csv("../matrices_and_networks/primary_subjects/3_community_policies/edgelist_3.csv")

In [903]:
# Visualize the edgelist for all bills whose primary subject is 3 (among any other possible primary subjects)
edgelist_3

,from,to,weight
0,Charles GOERENS (LUX-RENEW),Peter LIESE (DEU-EPP),1
1,Charles GOERENS (LUX-RENEW),María Soraya RODRÍGUEZ RAMOS (ESP-RENEW),1
2,Charles GOERENS (LUX-RENEW),Antonius MANDERS (NLD-EPP),1
3,Charles GOERENS (LUX-RENEW),Fabienne KELLER (FRA-RENEW),1
4,Charles GOERENS (LUX-RENEW),Sophia IN 'T VELD (NLD-RENEW),1
...,...,...,...
36650,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
36651,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
36652,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
36653,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [904]:
# Prepare the edges file of all bills whose primary subject is 3 (among any other possible primary subjects)
edges_3 = create_gephi_edges_file(edgelist_3)

# Export the edges file as a CSV
edges_3.to_csv("../matrices_and_networks/primary_subjects/3_community_policies/edges.csv", index=False)

In [905]:
# Filter MEPs present in either Source or Target in edges_3
unique_mep_ids = set(edges_3['Source']).union(set(edges_3['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/primary_subjects/3_community_policies/nodes.csv", index=False)

### 3.4. Economic, social and territorial cohesion
<a id="Economic-social-and-territorial-cohesion"></a>

In [906]:
# Filter for only bills whose primary subject is 4 (among any other possible primary subjects)
primary_subject_bills_4 = primary_subject_bills[primary_subject_bills["primary_subject"] == 4]

In [907]:
# Filter the member_votes dataset to include only voting records corresponding to bills whose primary subject is 4 (among any other possible primary subjects)
member_votes_4 = member_votes[member_votes['id_bill'].isin(primary_subject_bills_4['id_bill'])]

In [908]:
# Create the agents and artifacts matrix for all bills whose primary subject is 4 (among any other possible primary subjects)
all_bills_primary_subject_4 = create_agents_and_artifacts_matrix(member_votes_4)

all_bills_primary_subject_4

,name,bill_5,bill_12,bill_19,bill_28,bill_38,bill_44,bill_49,bill_52,bill_64,...,bill_1865,bill_1866,bill_1867,bill_1876,bill_1877,bill_1879,bill_1883,bill_1884,bill_1895,bill_1902
0,Charles GOERENS (LUX-RENEW),1,1,1,1,1,1,1,0,0,...,0,1,1,1,1,1,1,1,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),1,0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,0,1
4,Bernd LANGE (DEU-SD),1,1,1,1,1,1,1,0,0,...,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,0,1,0,0
864,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,1,1,0,0
865,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,0,0
866,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,0,1


In [909]:
# Save the agents and artifacts matrix for all bills whose primary subject is 4 (among any other possible primary subjects)
all_bills_primary_subject_4.to_csv("../matrices_and_networks/primary_subjects/4_economic_social_and_territorial_cohesion/agents_and_artifacts_4.csv", index=False)

In [910]:
# Load the edgelist for all bills whose primary subject is 4 (among any other possible primary subjects)
edgelist_4 = pd.read_csv("../matrices_and_networks/primary_subjects/4_economic_social_and_territorial_cohesion/edgelist_4.csv")

In [911]:
# Visualize the edgelist for all bills whose primary subject is 4 (among any other possible primary subjects)
edgelist_4

,from,to,weight
0,Bill NEWTON DUNN (GBR-RENEW),Richard CORBETT (GBR-SD),1
1,Bill NEWTON DUNN (GBR-RENEW),Geoffrey VAN ORDEN (GBR-ECR),1
2,Bill NEWTON DUNN (GBR-RENEW),Claude MORAES (GBR-SD),1
3,Bill NEWTON DUNN (GBR-RENEW),Neena GILL (GBR-SD),1
4,Bill NEWTON DUNN (GBR-RENEW),Jill EVANS (GBR-GREEN_EFA),1
...,...,...,...
20282,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
20283,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
20284,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
20285,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [912]:
# Prepare the edges file of all bills whose primary subject is 4 (among any other possible primary subjects)
edges_4 = create_gephi_edges_file(edgelist_4)

# Export the edges file as a CSV
edges_4.to_csv("../matrices_and_networks/primary_subjects/4_economic_social_and_territorial_cohesion/edges.csv", index=False)

In [913]:
# Filter MEPs present in either Source or Target in edges_4
unique_mep_ids = set(edges_4['Source']).union(set(edges_4['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/primary_subjects/4_economic_social_and_territorial_cohesion/nodes.csv", index=False)

### 3.5. Economic and monetary system
<a id="Economic-and-monetary-system"></a>

In [914]:
# Filter for only bills whose primary subject is 5 (among any other possible primary subjects)
primary_subject_bills_5 = primary_subject_bills[primary_subject_bills["primary_subject"] == 5]

In [915]:
# Filter the member_votes dataset to include only voting records corresponding to bills whose primary subject is 5 (among any other possible primary subjects)
member_votes_5 = member_votes[member_votes['id_bill'].isin(primary_subject_bills_5['id_bill'])]

In [916]:
# Create the agents and artifacts matrix for all bills whose primary subject is 5 (among any other possible primary subjects)
all_bills_primary_subject_5 = create_agents_and_artifacts_matrix(member_votes_5)

all_bills_primary_subject_5

,name,bill_19,bill_28,bill_112,bill_306,bill_307,bill_420,bill_437,bill_448,bill_470,...,bill_1372,bill_1624,bill_1652,bill_1676,bill_1707,bill_1723,bill_1853,bill_1870,bill_1871,bill_1872
0,Charles GOERENS (LUX-RENEW),1,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),1,1,1,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,1,1,1,1
4,Bernd LANGE (DEU-SD),1,1,1,1,1,1,0,1,1,...,1,0,1,1,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
863,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
864,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
865,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1


In [917]:
# Save the agents and artifacts matrix for all bills whose primary subject is 5 (among any other possible primary subjects)
all_bills_primary_subject_5.to_csv("../matrices_and_networks/primary_subjects/5_economic_and_monetary_system/agents_and_artifacts_5.csv", index=False)

In [918]:
# Load the edgelist for all bills whose primary subject is 5 (among any other possible primary subjects)
edgelist_5 = pd.read_csv("../matrices_and_networks/primary_subjects/5_economic_and_monetary_system/edgelist_5.csv")

In [919]:
# Visualize the edgelist for all bills whose primary subject is 5 (among any other possible primary subjects)
edgelist_5

,from,to,weight
0,Constanze KREHL (DEU-SD),Pierfrancesco MAJORINO (ITA-SD),1
1,Constanze KREHL (DEU-SD),Manuel PIZARRO (PRT-SD),1
2,Carlos COELHO (PRT-EPP),Thomas RUDNER (DEU-SD),1
3,Carlos COELHO (PRT-EPP),Anja HAGA (NLD-EPP),1
4,Carlos COELHO (PRT-EPP),Laura BALLARÍN CEREZA (ESP-SD),1
...,...,...,...
603,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
604,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
605,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
606,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [920]:
# Prepare the edges file of all bills whose primary subject is 5 (among any other possible primary subjects)
edges_5 = create_gephi_edges_file(edgelist_5)

# Export the edges file as a CSV
edges_5.to_csv("../matrices_and_networks/primary_subjects/5_economic_and_monetary_system/edges.csv", index=False)

In [921]:
# Filter MEPs present in either Source or Target in edges_5
unique_mep_ids = set(edges_5['Source']).union(set(edges_5['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/primary_subjects/5_economic_and_monetary_system/nodes.csv", index=False)

### 3.6. External relations of the Union
<a id="External-relations-of-the-Union"></a>

In [922]:
# Filter for only bills whose primary subject is 6 (among any other possible primary subjects)
primary_subject_bills_6 = primary_subject_bills[primary_subject_bills["primary_subject"] == 6]

In [923]:
# Filter the member_votes dataset to include only voting records corresponding to bills whose primary subject is 6 (among any other possible primary subjects)
member_votes_6 = member_votes[member_votes['id_bill'].isin(primary_subject_bills_6['id_bill'])]

In [924]:
# Create the agents and artifacts matrix for all bills whose primary subject is 6 (among any other possible primary subjects)
all_bills_primary_subject_6 = create_agents_and_artifacts_matrix(member_votes_6)

all_bills_primary_subject_6

,name,bill_1,bill_3,bill_12,bill_13,bill_14,bill_36,bill_39,bill_40,bill_41,...,bill_1852,bill_1856,bill_1876,bill_1880,bill_1895,bill_1898,bill_1899,bill_1900,bill_1901,bill_1902
0,Charles GOERENS (LUX-RENEW),1,1,1,0,1,0,1,0,0,...,1,1,1,1,1,1,1,1,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,1,1,1,1
4,Bernd LANGE (DEU-SD),1,0,1,0,1,0,1,0,0,...,0,1,1,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0
865,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,0
866,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0
867,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,1,1,1,1,1


In [925]:
# Save the agents and artifacts matrix for all bills whose primary subject is 6 (among any other possible primary subjects)
all_bills_primary_subject_6.to_csv("../matrices_and_networks/primary_subjects/6_external_relations_of_the_union/agents_and_artifacts_6.csv", index=False)

In [926]:
# Load the edgelist for all bills whose primary subject is 6 (among any other possible primary subjects)
edgelist_6 = pd.read_csv("../matrices_and_networks/primary_subjects/6_external_relations_of_the_union/edgelist_6.csv")

In [927]:
# Visualize the edgelist for all bills whose primary subject is 6 (among any other possible primary subjects)
edgelist_6

,from,to,weight
0,Charles GOERENS (LUX-RENEW),Sophia IN 'T VELD (NLD-RENEW),1
1,Charles GOERENS (LUX-RENEW),Nils TORVALDS (FIN-RENEW),1
2,Charles GOERENS (LUX-RENEW),Petras AUŠTREVIČIUS (LTU-RENEW),1
3,Charles GOERENS (LUX-RENEW),Nicola DANTI (ITA-RENEW),1
4,Charles GOERENS (LUX-RENEW),Marie-Pierre VEDRENNE (FRA-RENEW),1
...,...,...,...
23206,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
23207,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
23208,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
23209,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [928]:
# Prepare the edges file of all bills whose primary subject is 6 (among any other possible primary subjects)
edges_6 = create_gephi_edges_file(edgelist_6)

# Export the edges file as a CSV
edges_6.to_csv("../matrices_and_networks/primary_subjects/6_external_relations_of_the_union/edges.csv", index=False)

In [929]:
# Filter MEPs present in either Source or Target in edges_6
unique_mep_ids = set(edges_6['Source']).union(set(edges_6['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/primary_subjects/6_external_relations_of_the_union/nodes.csv", index=False)

### 3.7. Area of freedom, security and justice
<a id="Area-of-freedom-security-and-justice"></a>

In [930]:
# Filter for only bills whose primary subject is 7 (among any other possible primary subjects)
primary_subject_bills_7 = primary_subject_bills[primary_subject_bills["primary_subject"] == 7]

In [931]:
# Filter the member_votes dataset to include only voting records corresponding to bills whose primary subject is 7 (among any other possible primary subjects)
member_votes_7 = member_votes[member_votes['id_bill'].isin(primary_subject_bills_7['id_bill'])]

In [932]:
# Create the agents and artifacts matrix for all bills whose primary subject is 7 (among any other possible primary subjects)
all_bills_primary_subject_7 = create_agents_and_artifacts_matrix(member_votes_7)

all_bills_primary_subject_7

,name,bill_2,bill_4,bill_11,bill_15,bill_17,bill_37,bill_54,bill_64,bill_69,...,bill_1851,bill_1854,bill_1856,bill_1859,bill_1871,bill_1874,bill_1875,bill_1884,bill_1885,bill_1891
0,Charles GOERENS (LUX-RENEW),1,1,1,1,0,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),0,1,0,1,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,Bernd LANGE (DEU-SD),0,1,1,1,0,1,1,0,1,...,1,1,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,0,1,1,1
865,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,0,1,1,1
866,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,0,1,1,0,1,1,1
867,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [933]:
# Save the agents and artifacts matrix for all bills whose primary subject is 7 (among any other possible primary subjects)
all_bills_primary_subject_7.to_csv("../matrices_and_networks/primary_subjects/7_area_of_freedom_security_and_justice/agents_and_artifacts_7.csv", index=False)

In [934]:
# Load the edgelist for all bills whose primary subject is 7 (among any other possible primary subjects)
edgelist_7 = pd.read_csv("../matrices_and_networks/primary_subjects/7_area_of_freedom_security_and_justice/edgelist_7.csv")

In [935]:
# Visualize the edgelist for all bills whose primary subject is 7 (among any other possible primary subjects)
edgelist_7

,from,to,weight
0,Bill NEWTON DUNN (GBR-RENEW),Richard CORBETT (GBR-SD),1
1,Bill NEWTON DUNN (GBR-RENEW),Geoffrey VAN ORDEN (GBR-ECR),1
2,Bill NEWTON DUNN (GBR-RENEW),Claude MORAES (GBR-SD),1
3,Bill NEWTON DUNN (GBR-RENEW),Neena GILL (GBR-SD),1
4,Bill NEWTON DUNN (GBR-RENEW),Jill EVANS (GBR-GREEN_EFA),1
...,...,...,...
15046,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
15047,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
15048,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
15049,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [936]:
# Prepare the edges file of all bills whose primary subject is 7 (among any other possible primary subjects)
edges_7 = create_gephi_edges_file(edgelist_7)

# Export the edges file as a CSV
edges_7.to_csv("../matrices_and_networks/primary_subjects/7_area_of_freedom_security_and_justice/edges.csv", index=False)

In [937]:
# Filter MEPs present in either Source or Target in edges_7
unique_mep_ids = set(edges_7['Source']).union(set(edges_7['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/primary_subjects/7_area_of_freedom_security_and_justice/nodes.csv", index=False)

### 3.8. State and evolution of the Union
<a id="State-and-evolution-of-the-Union"></a>

In [938]:
# Filter for only bills whose primary subject is 8 (among any other possible primary subjects)
primary_subject_bills_8 = primary_subject_bills[primary_subject_bills["primary_subject"] == 8]

In [939]:
# Filter the member_votes dataset to include only voting records corresponding to bills whose primary subject is 8 (among any other possible primary subjects)
member_votes_8 = member_votes[member_votes['id_bill'].isin(primary_subject_bills_8['id_bill'])]

In [940]:
# Create the agents and artifacts matrix for all bills whose primary subject is 8 (among any other possible primary subjects)
all_bills_primary_subject_8 = create_agents_and_artifacts_matrix(member_votes_8)

all_bills_primary_subject_8

,name,bill_5,bill_6,bill_7,bill_8,bill_9,bill_10,bill_16,bill_17,bill_18,...,bill_1857,bill_1864,bill_1866,bill_1867,bill_1892,bill_1893,bill_1894,bill_1896,bill_1897,bill_1903
0,Charles GOERENS (LUX-RENEW),1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
1,Bill NEWTON DUNN (GBR-RENEW),1,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Constanze KREHL (DEU-SD),1,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Carlos COELHO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,1,1
4,Bernd LANGE (DEU-SD),1,1,1,1,1,1,1,0,1,...,1,0,0,0,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,Vasco BECKER-WEINBERG (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,0,0,0,0
864,Ana Miguel DOS SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,0,0,0,0
865,Vânia NETO (PRT-EPP),0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,0,0,0,0
866,Teófilo SANTOS (PRT-EPP),0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,1


In [941]:
# Save the agents and artifacts matrix for all bills whose primary subject is 8 (among any other possible primary subjects)
all_bills_primary_subject_8.to_csv("../matrices_and_networks/primary_subjects/8_state_and_evolution_of_the_union/agents_and_artifacts_8.csv", index=False)

In [942]:
# Load the edgelist for all bills whose primary subject is 8 (among any other possible primary subjects)
edgelist_8 = pd.read_csv("../matrices_and_networks/primary_subjects/8_state_and_evolution_of_the_union/edgelist_8.csv")

In [943]:
# Visualize the edgelist for all bills whose primary subject is 8 (among any other possible primary subjects)
edgelist_8

,from,to,weight
0,Bill NEWTON DUNN (GBR-RENEW),Evelyne GEBHARDT (DEU-SD),1
1,Bill NEWTON DUNN (GBR-RENEW),Richard CORBETT (GBR-SD),1
2,Bill NEWTON DUNN (GBR-RENEW),Geoffrey VAN ORDEN (GBR-ECR),1
3,Bill NEWTON DUNN (GBR-RENEW),Claude MORAES (GBR-SD),1
4,Bill NEWTON DUNN (GBR-RENEW),Neena GILL (GBR-SD),1
...,...,...,...
23221,Ana Miguel DOS SANTOS (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
23222,Ana Miguel DOS SANTOS (PRT-EPP),Ricardo MORGADO (PRT-EPP),1
23223,Vânia NETO (PRT-EPP),Teófilo SANTOS (PRT-EPP),1
23224,Vânia NETO (PRT-EPP),Ricardo MORGADO (PRT-EPP),1


In [944]:
# Prepare the edges file of all bills whose primary subject is 8 (among any other possible primary subjects)
edges_8 = create_gephi_edges_file(edgelist_8)

# Export the edges file as a CSV
edges_8.to_csv("../matrices_and_networks/primary_subjects/8_state_and_evolution_of_the_union/edges.csv", index=False)

In [945]:
# Filter MEPs present in either Source or Target in edges_8
unique_mep_ids = set(edges_8['Source']).union(set(edges_8['Target']))

# Filter group_memberships_single based on these MEP IDs
nodes_df = group_memberships_single[group_memberships_single['id_mep'].isin(unique_mep_ids)][['id_mep', 'group_code']]

# Save the filtered nodes dataframe to a CSV file in Gephi-compatible format
nodes_df.rename(columns={'id_mep': 'Id', 'group_code': 'Label'}, inplace=True)

# Add a "group_code" column that has the same values as the "Label" column to use as a partition in Gephi
nodes_df['group_code'] = nodes_df['Label']

# Save the nodes dataframe as a CSV file
nodes_df.to_csv("../matrices_and_networks/primary_subjects/8_state_and_evolution_of_the_union/nodes.csv", index=False)